In [1]:
import numpy as np
import pycbc.conversions as pc
import pycbc.pnutils as pnu
import pandas as pd
import h5py, os

from collections import defaultdict

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import random

import sys
from lal import YRJUL_SI

import seaborn as sns

import ast

In [2]:
def save_dict_to_hdf5(dic, filename):
    """
    ....
    """
    with h5py.File(filename, 'w') as h5file:
        recursively_save_dict_contents_to_group(h5file, '/', dic)

def recursively_save_dict_contents_to_group(h5file, path, dic):
    """
    ....
    """
    for key, item in dic.items():
        if isinstance(item, (np.ndarray, np.int64, np.float64, str, bytes)):
            h5file[path + key] = item
        elif isinstance(item, dict):
            recursively_save_dict_contents_to_group(h5file, path + key + '/', item)
        else:
            raise ValueError('Cannot save %s type'%type(item))

def load_dict_from_hdf5(filename):
    """
    ....
    """
    with h5py.File(filename, 'r') as h5file:
        return recursively_load_dict_contents_from_group(h5file, '/')

def recursively_load_dict_contents_from_group(h5file, path):
    """
    ....
    """
    ans = {}
    for key, item in h5file[path].items():
        if isinstance(item, h5py._hl.dataset.Dataset):
            ans[key] = item[()]
        elif isinstance(item, h5py._hl.group.Group):
            ans[key] = recursively_load_dict_contents_from_group(h5file, path + key + '/')
            
    return ans


def print_some_rcParams():
    rc_keys = ['backend', 'axes.labelsize', 'axes.titlesize', 'font.size',
        'legend.fontsize', 'xtick.labelsize', 'ytick.labelsize', 'font.family',
        'text.usetex', 'figure.figsize'] # 'font.sans-serif', 'font.serif'
    print('Using the following matplotlib settings:')
    for k in rc_keys:
        print(k, ':', plt.rcParams[k])

In [3]:
ifar_threshold = 10 ## yr 
number_injection = 10

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_columns', None)

In [5]:
#This is the first file, all events are true
injection_file = '/home/bhooshan.gadre/work/O3/sub_solar_search/O3B_VTs/ssm-search/O3b_VTs/common_ssm_injections.hdf'
injections = load_dict_from_hdf5(injection_file)
injections.keys(), len(injections.keys())

(dict_keys(['coa_phase', 'declination', 'distance', 'gstlal_ifar', 'gstlal_weight', 'inclination', 'mass1_source', 'mass2_source', 'mbta_ifar', 'polarization', 'pycbc_ifar', 'right_ascension', 'snr_H', 'snr_L', 'snr_V', 'spin1x', 'spin1y', 'spin1z', 'spin2x', 'spin2y', 'spin2z', 'time']),
 22)

In [6]:
df_true = pd.DataFrame(injections)
df_true

,coa_phase,declination,distance,gstlal_ifar,gstlal_weight,inclination,mass1_source,mass2_source,mbta_ifar,polarization,pycbc_ifar,right_ascension,snr_H,snr_L,snr_V,spin1x,spin1y,spin1z,spin2x,spin2y,spin2z,time
0,1.463821,-1.232698,231.483117,0.0,0.666667,2.867627,1.692460,0.501600,0.0,2.515262,0.0,0.164688,2.440571,2.957571,1.361124,0.053696,0.053141,-0.037187,0.057080,-0.011039,0.007150,1.238166e+09
1,0.673478,-0.622410,458.599031,0.0,1.000000,0.918732,6.421466,0.763863,0.0,0.135494,0.0,2.177941,2.684150,2.988815,1.259368,-0.025080,-0.011637,0.021298,0.020269,0.071086,0.038439,1.238166e+09
2,5.447231,-1.440067,302.357273,0.0,0.666667,0.854144,10.481894,1.045793,0.0,0.132165,0.0,2.621765,4.408046,6.731172,1.315282,-0.004171,-0.046270,0.012784,0.021928,-0.027524,-0.030501,1.238166e+09
3,5.521864,-1.276307,463.341251,0.0,0.666667,0.494481,8.906005,0.811978,0.0,1.828451,0.0,3.177169,2.049177,3.539541,0.762279,0.000633,0.001265,-0.000504,0.004547,-0.011592,-0.004068,1.238166e+09
4,0.183151,-0.734805,833.927353,0.0,1.000000,2.895566,7.129897,0.808812,0.0,2.805147,0.0,2.219308,2.389652,3.215351,0.549881,-0.051269,0.011986,-0.001127,-0.021172,0.014852,-0.003302,1.238166e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1916578,0.187059,-0.469964,752.284246,0.0,1.000000,0.388063,5.784159,0.559250,0.0,3.122078,0.0,4.757083,2.959844,2.958614,0.812961,-0.032232,0.036856,-0.038265,-0.001476,0.035159,-0.032295,1.269562e+09
1916579,0.141624,-0.272180,643.259480,0.0,0.666667,3.032152,8.621294,0.868851,0.0,0.817515,0.0,1.586071,2.718237,3.586084,3.759479,-0.007036,-0.019932,0.032837,-0.038478,0.020862,-0.010397,1.269562e+09
1916580,1.174031,-0.227858,349.130344,0.0,0.666667,2.206089,10.560706,0.717379,0.0,1.318691,0.0,3.475683,3.402718,2.469075,1.012893,0.004500,-0.054032,-0.029833,0.062644,0.023295,-0.031378,1.269562e+09
1916581,5.752044,0.708430,455.222188,0.0,1.000000,2.115026,5.727628,0.886952,0.0,1.553802,0.0,2.464774,2.854940,3.862371,0.634093,-0.028952,-0.016063,-0.024699,-0.048624,-0.022375,0.007844,1.269562e+09


In [7]:
df_true = df_true.drop(columns=['coa_phase', 'declination', 'distance', 'gstlal_ifar', 'gstlal_weight', 'inclination', 'mbta_ifar', 'polarization', 
                                'right_ascension', 'spin1x', 'spin1y', 'spin2x', 'spin2y', 'time'])


df_true = df_true.rename(columns={'mass2_source': 'mass2', 
                                  'mass1_source': 'mass1',
                                  'snr_H': 'snr_H1',
                                  'snr_L': 'snr_L1',
                                  'snr_V': 'snr_V1'
                                 })
#df_true['label'] = 1

df_true

,mass1,mass2,pycbc_ifar,snr_H1,snr_L1,snr_V1,spin1z,spin2z
0,1.692460,0.501600,0.0,2.440571,2.957571,1.361124,-0.037187,0.007150
1,6.421466,0.763863,0.0,2.684150,2.988815,1.259368,0.021298,0.038439
2,10.481894,1.045793,0.0,4.408046,6.731172,1.315282,0.012784,-0.030501
3,8.906005,0.811978,0.0,2.049177,3.539541,0.762279,-0.000504,-0.004068
4,7.129897,0.808812,0.0,2.389652,3.215351,0.549881,-0.001127,-0.003302
...,...,...,...,...,...,...,...,...
1916578,5.784159,0.559250,0.0,2.959844,2.958614,0.812961,-0.038265,-0.032295
1916579,8.621294,0.868851,0.0,2.718237,3.586084,3.759479,0.032837,-0.010397
1916580,10.560706,0.717379,0.0,3.402718,2.469075,1.012893,-0.029833,-0.031378
1916581,5.727628,0.886952,0.0,2.854940,3.862371,0.634093,-0.024699,0.007844


In [8]:
opt_snr = df_true['snr_H1']**2. + df_true['snr_L1']**2. + df_true['snr_V1']**2. 
df_true['opt_snr'] = np.sqrt(opt_snr) 
df_true

,mass1,mass2,pycbc_ifar,snr_H1,snr_L1,snr_V1,spin1z,spin2z,opt_snr
0,1.692460,0.501600,0.0,2.440571,2.957571,1.361124,-0.037187,0.007150,4.068939
1,6.421466,0.763863,0.0,2.684150,2.988815,1.259368,0.021298,0.038439,4.209951
2,10.481894,1.045793,0.0,4.408046,6.731172,1.315282,0.012784,-0.030501,8.152884
3,8.906005,0.811978,0.0,2.049177,3.539541,0.762279,-0.000504,-0.004068,4.160354
4,7.129897,0.808812,0.0,2.389652,3.215351,0.549881,-0.001127,-0.003302,4.043672
...,...,...,...,...,...,...,...,...,...
1916578,5.784159,0.559250,0.0,2.959844,2.958614,0.812961,-0.038265,-0.032295,4.263212
1916579,8.621294,0.868851,0.0,2.718237,3.586084,3.759479,0.032837,-0.010397,5.863658
1916580,10.560706,0.717379,0.0,3.402718,2.469075,1.012893,-0.029833,-0.031378,4.324439
1916581,5.727628,0.886952,0.0,2.854940,3.862371,0.634093,-0.024699,0.007844,4.844654


In [9]:
#this the background
trig_file = "/home/pycbc.offline/O3/production/O3A_SUB_SOLAR/sub_solar/STATMAP/H1L1V1-COMBINE_STATMAP_FULL_DATA-1238787802-853441.hdf"
bank_file = "/home/pycbc.offline/O3/production/O3A_SUB_SOLAR/sub_solar/BANK/H1L1V1-BANK2HDF-1238787802-853441.hdf"

In [10]:
%%time
bank = h5py.File(bank_file, 'r')
hashes = bank['template_hash'][:]

bank_tids = hashes.argsort()
fields = ['mass1', 'mass2', 'spin1z', 'spin2z']
trigs = h5py.File(trig_file, 'r')

trig_params = defaultdict(list)
for tid in trigs['background']['template_id'][:number_injection]:  ## test with 10 triggers. Later this can be [:] for all triggers
    bank_idx = np.argwhere(bank_tids == tid)[0]
    for k in fields:
        trig_params[k].append(bank[k][:][bank_idx][0])
#print(trig_params)

CPU times: user 281 ms, sys: 151 ms, total: 432 ms
Wall time: 738 ms


In [11]:
df_false = pd.read_csv('background_SNR.txt', sep='\t')
df_false

,stat,ifar,time_H1,tid_H1,coa_phase_H1,snr_H1,time_L1,tid_L1,coa_phase_L1,snr_L1,time_V1,tid_V1,coa_phase_V1,snr_V1,mean_time,mass1,mass2,spin1z,spin2z
0,4.183380,NaN,1.241128e+09,302933848,2.761589,5.232667,1.241308e+09,338162294,-0.443711,4.995651,-1.0,-1,-1.0,-1.0,1.241218e+09,1.805386,0.227351,-0.336657,-0.031896
1,7.727292,NaN,1.241423e+09,37374621,-2.771329,5.738260,1.241253e+09,41722153,0.332370,5.382648,-1.0,-1,-1.0,-1.0,1.241338e+09,0.868567,0.212223,-0.897744,0.083385
2,11.296075,NaN,1.241582e+09,663440558,-1.804354,6.424503,1.241316e+09,740543038,0.041965,5.420436,-1.0,-1,-1.0,-1.0,1.241449e+09,1.073055,0.237231,0.748448,-0.080374
3,15.634021,NaN,1.241560e+09,1954222950,3.132144,5.128339,1.241516e+09,2181471735,-0.281314,6.999949,-1.0,-1,-1.0,-1.0,1.241538e+09,0.512530,0.214569,0.324258,0.036742
4,7.796479,NaN,1.241662e+09,331406446,1.282000,5.371046,1.241503e+09,369945869,-1.666816,5.187170,-1.0,-1,-1.0,-1.0,1.241583e+09,0.511975,0.201500,-0.425553,0.055912
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1916578,11.014588,NaN,1.241268e+09,993387280,-1.674996,5.696897,1.241524e+09,1108884002,1.515363,5.719054,-1.0,-1,-1.0,-1.0,1.241396e+09,0.691660,0.217300,-0.611034,-0.041273
1916579,10.270576,NaN,1.241639e+09,1890461227,-2.662703,5.279950,1.241662e+09,2110300667,0.573829,6.158387,-1.0,-1,-1.0,-1.0,1.241651e+09,1.076801,0.215017,-0.737290,-0.025544
1916580,15.361928,NaN,1.241132e+09,31148581,-1.880110,5.734617,1.241620e+09,34770672,0.910409,6.537734,-1.0,-1,-1.0,-1.0,1.241376e+09,1.391555,0.211880,-0.714896,-0.051568
1916581,8.121799,NaN,1.241135e+09,715224250,1.796237,5.669559,1.241169e+09,798343013,-2.095810,5.611607,-1.0,-1,-1.0,-1.0,1.241152e+09,4.957479,0.856356,-0.226959,-0.819073


In [12]:
df_false = df_false.drop(columns=['stat', 'ifar', 'time_H1', 'tid_H1', 'time_L1', 'tid_L1', 'time_V1', 'tid_V1', 
                                'mean_time', 'coa_phase_V1', 'coa_phase_H1','coa_phase_L1'])

#df_false['label'] = 0

df_false

,snr_H1,snr_L1,snr_V1,mass1,mass2,spin1z,spin2z
0,5.232667,4.995651,-1.0,1.805386,0.227351,-0.336657,-0.031896
1,5.738260,5.382648,-1.0,0.868567,0.212223,-0.897744,0.083385
2,6.424503,5.420436,-1.0,1.073055,0.237231,0.748448,-0.080374
3,5.128339,6.999949,-1.0,0.512530,0.214569,0.324258,0.036742
4,5.371046,5.187170,-1.0,0.511975,0.201500,-0.425553,0.055912
...,...,...,...,...,...,...,...
1916578,5.696897,5.719054,-1.0,0.691660,0.217300,-0.611034,-0.041273
1916579,5.279950,6.158387,-1.0,1.076801,0.215017,-0.737290,-0.025544
1916580,5.734617,6.537734,-1.0,1.391555,0.211880,-0.714896,-0.051568
1916581,5.669559,5.611607,-1.0,4.957479,0.856356,-0.226959,-0.819073


In [13]:
opt_snr = df_false['snr_H1']**2. + df_false['snr_L1']**2. + df_false['snr_V1']**2. 
df_false['opt_snr'] = np.sqrt(opt_snr) 
df_false

,snr_H1,snr_L1,snr_V1,mass1,mass2,spin1z,spin2z,opt_snr
0,5.232667,4.995651,-1.0,1.805386,0.227351,-0.336657,-0.031896,7.303241
1,5.738260,5.382648,-1.0,0.868567,0.212223,-0.897744,0.083385,7.930985
2,6.424503,5.420436,-1.0,1.073055,0.237231,0.748448,-0.080374,8.464949
3,5.128339,6.999949,-1.0,0.512530,0.214569,0.324258,0.036742,8.734938
4,5.371046,5.187170,-1.0,0.511975,0.201500,-0.425553,0.055912,7.533583
...,...,...,...,...,...,...,...,...
1916578,5.696897,5.719054,-1.0,0.691660,0.217300,-0.611034,-0.041273,8.134015
1916579,5.279950,6.158387,-1.0,1.076801,0.215017,-0.737290,-0.025544,8.173347
1916580,5.734617,6.537734,-1.0,1.391555,0.211880,-0.714896,-0.051568,8.753730
1916581,5.669559,5.611607,-1.0,4.957479,0.856356,-0.226959,-0.819073,8.039530


In [14]:
#df_false = pd.DataFrame(trig_params)

df_false['pycbc_ifar'] = trigs['background_h0']['H1L1']['ifar'][:1916583]

df_false

,snr_H1,snr_L1,snr_V1,mass1,mass2,spin1z,spin2z,opt_snr,pycbc_ifar
0,5.232667,4.995651,-1.0,1.805386,0.227351,-0.336657,-0.031896,7.303241,0.000314
1,5.738260,5.382648,-1.0,0.868567,0.212223,-0.897744,0.083385,7.930985,0.000070
2,6.424503,5.420436,-1.0,1.073055,0.237231,0.748448,-0.080374,8.464949,0.000018
3,5.128339,6.999949,-1.0,0.512530,0.214569,0.324258,0.036742,8.734938,0.000821
4,5.371046,5.187170,-1.0,0.511975,0.201500,-0.425553,0.055912,7.533583,0.000288
...,...,...,...,...,...,...,...,...,...
1916578,5.696897,5.719054,-1.0,0.691660,0.217300,-0.611034,-0.041273,8.134015,0.000010
1916579,5.279950,6.158387,-1.0,1.076801,0.215017,-0.737290,-0.025544,8.173347,0.000011
1916580,5.734617,6.537734,-1.0,1.391555,0.211880,-0.714896,-0.051568,8.753730,0.000470
1916581,5.669559,5.611607,-1.0,4.957479,0.856356,-0.226959,-0.819073,8.039530,0.000045


In [15]:
df = pd.concat([df_true, df_false], axis=0)
df

,mass1,mass2,pycbc_ifar,snr_H1,snr_L1,snr_V1,spin1z,spin2z,opt_snr
0,1.692460,0.501600,0.000000,2.440571,2.957571,1.361124,-0.037187,0.007150,4.068939
1,6.421466,0.763863,0.000000,2.684150,2.988815,1.259368,0.021298,0.038439,4.209951
2,10.481894,1.045793,0.000000,4.408046,6.731172,1.315282,0.012784,-0.030501,8.152884
3,8.906005,0.811978,0.000000,2.049177,3.539541,0.762279,-0.000504,-0.004068,4.160354
4,7.129897,0.808812,0.000000,2.389652,3.215351,0.549881,-0.001127,-0.003302,4.043672
...,...,...,...,...,...,...,...,...,...
1916578,0.691660,0.217300,0.000010,5.696897,5.719054,-1.000000,-0.611034,-0.041273,8.134015
1916579,1.076801,0.215017,0.000011,5.279950,6.158387,-1.000000,-0.737290,-0.025544,8.173347
1916580,1.391555,0.211880,0.000470,5.734617,6.537734,-1.000000,-0.714896,-0.051568,8.753730
1916581,4.957479,0.856356,0.000045,5.669559,5.611607,-1.000000,-0.226959,-0.819073,8.039530


In [16]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
import matplotlib.pyplot as plt
import seaborn as sns

In [17]:
# Data exploration
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3833166 entries, 0 to 1916582
Data columns (total 9 columns):
 #   Column      Dtype  
---  ------      -----  
 0   mass1       float64
 1   mass2       float64
 2   pycbc_ifar  float64
 3   snr_H1      float64
 4   snr_L1      float64
 5   snr_V1      float64
 6   spin1z      float64
 7   spin2z      float64
 8   opt_snr     float64
dtypes: float64(9)
memory usage: 292.4 MB


In [18]:
df.describe()

,mass1,mass2,pycbc_ifar,snr_H1,snr_L1,snr_V1,spin1z,spin2z,opt_snr
count,3.833166e+06,3.833166e+06,3.833166e+06,3.833166e+06,3.833166e+06,3.833166e+06,3.833166e+06,3.833166e+06,3.833166e+06
mean,3.434631e+00,4.882454e-01,3.175995e+02,4.280818e+00,4.950683e+00,7.838706e-01,-7.084968e-02,4.095302e-03,7.150476e+00
std,3.114635e+00,2.847178e-01,3.890053e+03,2.298016e+00,2.411279e+00,2.121235e+00,3.859664e-01,1.242754e-01,2.813287e+00
min,1.958331e-01,1.900007e-01,0.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-9.000000e-01,-8.999995e-01,4.000000e+00
25%,8.600363e-01,2.329284e-01,0.000000e+00,2.889139e+00,3.705169e+00,-1.000000e+00,-1.153297e-01,-3.033717e-02,5.107287e+00
50%,2.065527e+00,3.765660e-01,1.548821e-06,4.990977e+00,5.308345e+00,5.991942e-01,-5.767238e-03,1.281223e-03,7.701957e+00
75%,5.632518e+00,7.236376e-01,4.354362e-05,5.587894e+00,5.889004e+00,1.392704e+00,3.467941e-02,3.862756e-02,8.292352e+00
max,1.099997e+01,1.100000e+00,1.427988e+05,3.016193e+02,4.439963e+02,1.548895e+02,9.000000e-01,8.999998e-01,5.402225e+02


# Model with all data

In [19]:
X = df.drop('pycbc_ifar', axis=1)
y = df['pycbc_ifar']

In [20]:
X

,mass1,mass2,snr_H1,snr_L1,snr_V1,spin1z,spin2z,opt_snr
0,1.692460,0.501600,2.440571,2.957571,1.361124,-0.037187,0.007150,4.068939
1,6.421466,0.763863,2.684150,2.988815,1.259368,0.021298,0.038439,4.209951
2,10.481894,1.045793,4.408046,6.731172,1.315282,0.012784,-0.030501,8.152884
3,8.906005,0.811978,2.049177,3.539541,0.762279,-0.000504,-0.004068,4.160354
4,7.129897,0.808812,2.389652,3.215351,0.549881,-0.001127,-0.003302,4.043672
...,...,...,...,...,...,...,...,...
1916578,0.691660,0.217300,5.696897,5.719054,-1.000000,-0.611034,-0.041273,8.134015
1916579,1.076801,0.215017,5.279950,6.158387,-1.000000,-0.737290,-0.025544,8.173347
1916580,1.391555,0.211880,5.734617,6.537734,-1.000000,-0.714896,-0.051568,8.753730
1916581,4.957479,0.856356,5.669559,5.611607,-1.000000,-0.226959,-0.819073,8.039530


In [21]:
y

0          0.000000
1          0.000000
2          0.000000
3          0.000000
4          0.000000
             ...   
1916578    0.000010
1916579    0.000011
1916580    0.000470
1916581    0.000045
1916582    0.000006
Name: pycbc_ifar, Length: 3833166, dtype: float64

In [22]:
def train_random_forest(X, y, n_estimators, ifar_threshold=10):
    # Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Report the current n_estimators
    print(f"Entrenando modelo con n_estimators={n_estimators}")

    # Create and train the model
    model = RandomForestRegressor(n_estimators=n_estimators, random_state=42)
    model.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = model.predict(X_test)
    
    # Calculate metrics
    mse = mean_squared_error(y_test, y_pred)
    rmse = mse ** 0.5
    mae = mean_absolute_error(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # Calculate the custom prediction
    prediction_value = (y_pred > ifar_threshold).sum() / len(y_test) * 100
    
    # Show results with specific comments from n_estimators
    print(f"len(y_test) con n_estimators={n_estimators}: {len(y_test)}")
    print(f"(y_test > 10).sum() con n_estimators={n_estimators}: {(y_test > 10).sum()}")
    print(f"(y_pred > 10).sum() con n_estimators={n_estimators}: {(y_pred > 10).sum()}")
    print(f"predicción con n_estimators={n_estimators}: {prediction_value:.2f}%")
    print(f"MSE con n_estimators={n_estimators}: {mse:.4f}")
    print(f"RMSE con n_estimators={n_estimators}: {rmse:.4f}")
    print(f"MAE con n_estimators={n_estimators}: {mae:.4f}")
    print(f"MAPE con n_estimators={n_estimators}: {mape:.4f}")
    print(f"R^2 con n_estimators={n_estimators}: {r2:.4f}")
    print("-" * 40)

    # Return the model and metrics if necessary
    return model, mse, rmse, mae, mape, r2, prediction_value


In [23]:
train_random_forest(X, y, 50, ifar_threshold=10)

Entrenando modelo con n_estimators=50
len(y_test) con n_estimators=50: 766634
(y_test > 10).sum() con n_estimators=50: 12800
(y_pred > 10).sum() con n_estimators=50: 295641
predicción con n_estimators=50: 38.56%
MSE con n_estimators=50: 15481149.2758
RMSE con n_estimators=50: 3934.6092
MAE con n_estimators=50: 678.7939
MAPE con n_estimators=50: 1656300521361058304.0000
R^2 con n_estimators=50: -0.0268
----------------------------------------


(RandomForestRegressor(n_estimators=50, random_state=42),
 15481149.275839165,
 3934.6091643057975,
 678.7939260704802,
 1.6563005213610583e+18,
 -0.02683670816438899,
 38.56351270619357)

# Model with df_true only

In [24]:
X = df_true.drop('pycbc_ifar', axis=1)
y = df_true['pycbc_ifar']

In [25]:
X

,mass1,mass2,snr_H1,snr_L1,snr_V1,spin1z,spin2z,opt_snr
0,1.692460,0.501600,2.440571,2.957571,1.361124,-0.037187,0.007150,4.068939
1,6.421466,0.763863,2.684150,2.988815,1.259368,0.021298,0.038439,4.209951
2,10.481894,1.045793,4.408046,6.731172,1.315282,0.012784,-0.030501,8.152884
3,8.906005,0.811978,2.049177,3.539541,0.762279,-0.000504,-0.004068,4.160354
4,7.129897,0.808812,2.389652,3.215351,0.549881,-0.001127,-0.003302,4.043672
...,...,...,...,...,...,...,...,...
1916578,5.784159,0.559250,2.959844,2.958614,0.812961,-0.038265,-0.032295,4.263212
1916579,8.621294,0.868851,2.718237,3.586084,3.759479,0.032837,-0.010397,5.863658
1916580,10.560706,0.717379,3.402718,2.469075,1.012893,-0.029833,-0.031378,4.324439
1916581,5.727628,0.886952,2.854940,3.862371,0.634093,-0.024699,0.007844,4.844654


In [26]:
train_random_forest(X, y, 50, ifar_threshold=10)

Entrenando modelo con n_estimators=50
len(y_test) con n_estimators=50: 383317
(y_test > 10).sum() con n_estimators=50: 12707
(y_pred > 10).sum() con n_estimators=50: 295432
predicción con n_estimators=50: 77.07%
MSE con n_estimators=50: 31002371.8045
RMSE con n_estimators=50: 5567.9774
MAE con n_estimators=50: 1353.1792
MAPE con n_estimators=50: 3331215210532269568.0000
R^2 con n_estimators=50: -0.0334
----------------------------------------


(RandomForestRegressor(n_estimators=50, random_state=42),
 31002371.804548956,
 5567.97735309232,
 1353.1792353087612,
 3.3312152105322696e+18,
 -0.03342806025097955,
 77.07250135005752)

# Model with df_true['ifar']>10 only

In [35]:
df_true = df_true[df_true['pycbc_ifar'] > 10]
df_true

,mass1,mass2,pycbc_ifar,snr_H1,snr_L1,snr_V1,spin1z,spin2z,opt_snr
17,10.044697,0.771064,10174.474264,3.236697,3.959241,0.672289,0.003799,-0.034952,5.157884
39,9.871082,0.608848,10174.474264,4.944136,4.885376,2.291932,0.003390,0.002160,7.318766
45,6.012620,0.290126,10174.474264,3.138401,2.989728,0.665509,0.022516,0.025649,4.385309
52,1.566219,0.522978,10174.474264,3.554345,1.999399,1.121289,0.032162,-0.004396,4.229451
59,5.331335,0.458099,10174.474264,5.381416,5.130757,1.040995,-0.004934,0.009750,7.507861
...,...,...,...,...,...,...,...,...,...
1903266,6.626040,0.982731,69.997932,2.532026,3.325533,0.341978,-0.030900,-0.025199,4.193718
1903269,10.661847,0.462810,25146.646963,4.366187,6.194481,2.369776,0.011288,0.066585,7.940468
1903305,2.810979,0.813847,54.111754,6.184364,6.846928,3.077704,0.012777,-0.049325,9.726204
1903359,8.898606,0.641294,25146.646963,2.951985,3.437002,0.377267,-0.067872,-0.015317,4.546376


In [36]:
X = df_true.drop('pycbc_ifar', axis=1)
y = df_true['pycbc_ifar']

In [37]:
X

,mass1,mass2,snr_H1,snr_L1,snr_V1,spin1z,spin2z,opt_snr
17,10.044697,0.771064,3.236697,3.959241,0.672289,0.003799,-0.034952,5.157884
39,9.871082,0.608848,4.944136,4.885376,2.291932,0.003390,0.002160,7.318766
45,6.012620,0.290126,3.138401,2.989728,0.665509,0.022516,0.025649,4.385309
52,1.566219,0.522978,3.554345,1.999399,1.121289,0.032162,-0.004396,4.229451
59,5.331335,0.458099,5.381416,5.130757,1.040995,-0.004934,0.009750,7.507861
...,...,...,...,...,...,...,...,...
1903266,6.626040,0.982731,2.532026,3.325533,0.341978,-0.030900,-0.025199,4.193718
1903269,10.661847,0.462810,4.366187,6.194481,2.369776,0.011288,0.066585,7.940468
1903305,2.810979,0.813847,6.184364,6.846928,3.077704,0.012777,-0.049325,9.726204
1903359,8.898606,0.641294,2.951985,3.437002,0.377267,-0.067872,-0.015317,4.546376


In [38]:
train_random_forest(X, y, 50, ifar_threshold=10)

Entrenando modelo con n_estimators=50
len(y_test) con n_estimators=50: 12663
(y_test > 10).sum() con n_estimators=50: 12663
(y_pred > 10).sum() con n_estimators=50: 12663
predicción con n_estimators=50: 100.00%
MSE con n_estimators=50: 580123572.5617
RMSE con n_estimators=50: 24085.7546
MAE con n_estimators=50: 18164.5581
MAPE con n_estimators=50: 74.1924
R^2 con n_estimators=50: -0.0481
----------------------------------------


(RandomForestRegressor(n_estimators=50, random_state=42),
 580123572.5616732,
 24085.75455661859,
 18164.558054068304,
 74.19237926674492,
 -0.04810662224896012,
 100.0)

# Model with df_true['ifar']>10 and  df_false['ifar']<10 only

In [39]:
df_false = df_false[df_false['pycbc_ifar'] < 10]

df_false = df_false.sample(n=63312, random_state=42)

df_false

,snr_H1,snr_L1,snr_V1,mass1,mass2,spin1z,spin2z,opt_snr,pycbc_ifar
847816,5.239635,6.653980,-1.000000,0.438831,0.240303,0.097319,0.026410,8.528143,0.000005
183932,5.183653,5.553951,-1.000000,0.218947,0.202394,-0.084089,0.092678,7.662678,0.000574
856136,-1.000000,5.637584,5.719407,1.516771,0.289223,-0.632653,0.090082,8.092834,0.000009
571603,5.369545,6.061606,-1.000000,1.436909,0.221619,-0.232619,-0.072837,8.159355,0.000942
1575046,5.156031,6.004288,-1.000000,1.070250,0.367832,-0.475623,0.091045,7.977226,0.000164
...,...,...,...,...,...,...,...,...,...
9232,5.729503,6.800716,-1.000000,0.704412,0.224493,-0.349655,-0.094622,8.948572,0.000011
19965,5.571110,5.832009,-1.000000,0.767730,0.226237,0.511055,0.011842,8.127090,0.000471
710446,5.208351,6.492634,-1.000000,1.110699,0.239793,-0.508516,-0.023408,8.383389,0.000016
1070609,5.433822,5.393438,-1.000000,0.379486,0.207273,0.034878,-0.084247,7.721114,0.001625


In [40]:
df = pd.concat([df_true, df_false], axis=0)
df

,mass1,mass2,pycbc_ifar,snr_H1,snr_L1,snr_V1,spin1z,spin2z,opt_snr
17,10.044697,0.771064,10174.474264,3.236697,3.959241,0.672289,0.003799,-0.034952,5.157884
39,9.871082,0.608848,10174.474264,4.944136,4.885376,2.291932,0.003390,0.002160,7.318766
45,6.012620,0.290126,10174.474264,3.138401,2.989728,0.665509,0.022516,0.025649,4.385309
52,1.566219,0.522978,10174.474264,3.554345,1.999399,1.121289,0.032162,-0.004396,4.229451
59,5.331335,0.458099,10174.474264,5.381416,5.130757,1.040995,-0.004934,0.009750,7.507861
...,...,...,...,...,...,...,...,...,...
9232,0.704412,0.224493,0.000011,5.729503,6.800716,-1.000000,-0.349655,-0.094622,8.948572
19965,0.767730,0.226237,0.000471,5.571110,5.832009,-1.000000,0.511055,0.011842,8.127090
710446,1.110699,0.239793,0.000016,5.208351,6.492634,-1.000000,-0.508516,-0.023408,8.383389
1070609,0.379486,0.207273,0.001625,5.433822,5.393438,-1.000000,0.034878,-0.084247,7.721114


In [41]:
X = df.drop('pycbc_ifar', axis=1)
y = df['pycbc_ifar']

In [42]:
train_random_forest(X, y, 50, ifar_threshold=10)

Entrenando modelo con n_estimators=50
len(y_test) con n_estimators=50: 25325
(y_test > 10).sum() con n_estimators=50: 12733
(y_pred > 10).sum() con n_estimators=50: 12830
predicción con n_estimators=50: 50.66%
MSE con n_estimators=50: 288149573.2122
RMSE con n_estimators=50: 16974.9690
MAE con n_estimators=50: 9101.8277
MAPE con n_estimators=50: 962658.4407
R^2 con n_estimators=50: 0.2070
----------------------------------------


(RandomForestRegressor(n_estimators=50, random_state=42),
 288149573.2121539,
 16974.969019475528,
 9101.827674780247,
 962658.440704789,
 0.20702169318790398,
 50.66140177690029)